In [1]:
import pandas as pd

In [2]:
# Read the Excel workbook
df = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls')

In [3]:
df.head()

,Food Environment Atlas data download
0,Notes about the Food Environment Atlas downloa...
1,This file contains multiple spreadsheets:
2,1. A variable list that includes metadata abo...
3,2. Spreadsheets that contain data for each of...
4,3. County and State-level supplemental data t...


In [18]:
# Load Access, Stores, Assistance, Insecurity, Local, Health, Restaurants, Socioeconomic data sheets as dataframes 

access = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'ACCESS')
stores = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'STORES')
assistance = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'ASSISTANCE')
insecurity = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'INSECURITY')
local = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'LOCAL')
health = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'HEALTH')
restaurants = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'RESTAURANTS')
socioeconomic = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'SOCIOECONOMIC')
prices_taxes = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'PRICES_TAXES')

In [19]:
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [22]:
restaurants.columns

Index([u'FIPS', u'State', u'County', u'FFR09', u'FFR14', u'PCH_FFR_09_14',
       u'FFRPTH09', u'FFRPTH14', u'PCH_FFRPTH_09_14', u'FSR09', u'FSR14',
       u'PCH_FSR_09_14', u'FSRPTH09', u'FSRPTH14', u'PCH_FSRPTH_09_14',
       u'PC_FFRSALES07', u'PC_FFRSALES12', u'PC_FSRSALES07', u'PC_FSRSALES12'],
      dtype='object')

In [21]:
restaurant_cols = ['FIPS', 'State', 'County', 'FFRPTH14', 'FSRPTH14']
stores_cols = ['FIPS', 'State', 'County', 'GROCPTH14', 'SUPERCPTH14', 'CONVSPTH14', 'SPECSPTH14', 'SNAPSPTH16']
access_cols = ['FIPS', 'State', 'County', 'PCT_LACCESS_POP15', 'PCT_LACCESS_LOWI15', 'PCT_LACCESS_HHNV15', 'PCT_LACCESS_SNAP15', 'PCT_LACCESS_CHILD15', 'PCT_LACCESS_SENIORS15', 'PCT_LACCESS_WHITE15', 'PCT_LACCESS_BLACK15', 'PCT_LACCESS_HISP15', 'PCT_LACCESS_NHASIAN15', 'PCT_LACCESS_NHNA15', 'PCT_LACCESS_NHPI15', 'PCT_LACCESS_MULTIR15']
assistance_cols = ['FIPS', 'State', 'County', 'PCT_SNAP16', 'PCT_NSLP15', 'PCT_SBP15', 'PCT_SFSP15', 'PCT_WIC15']
prices_taxes_cols = ['FIPS', 'State', 'County', 'SODATAX_STORES14', 'SODATAX_VENDM14', 'CHIPSTAX_STORES14', 'CHIPSTAX_VENDM14', 'FOOD_TAX14']
local_cols = ['FIPS', 'State', 'County', 'FMRKTPTH16', 'PCT_FMRKT_SNAP16', 'PCT_FMRKT_WIC16', 'PCT_FMRKT_WICCASH16', 'PCT_FMRKT_SFMNP16', 'PCT_FMRKT_CREDIT16', 'PCT_FMRKT_FRVEG16', 'PCT_FMRKT_ANMLPROD16', 'PCT_FMRKT_BAKED16', 'PCT_FMRKT_OTHERFOOD16', 'FOODHUB16']
health_cols = ['FIPS', 'State', 'County', 'PCT_DIABETES_ADULTS13', 'PCT_OBESE_ADULTS13', 'PCT_HSPA15', 'RECFACPTH14']
socioeconomic_cols = ['FIPS', 'State', 'County', 'PCT_NHWHITE10', 'PCT_NHBLACK10', 'PCT_HISP10', 'PCT_NHASIAN10', 'PCT_NHNA10', 'PCT_NHPI10', 'PCT_65OLDER10', 'PCT_18YOUNGER10', 'MEDHHINC15', 'POVRATE15', 'METRO13']


In [23]:
features_new = stores_cols + assistance_cols + local_cols + health_cols + restaurant_cols + socioeconomic_cols + prices_taxes_cols + access_cols
prediction_features = list(set(features_new) - set(['FIPS', 'State', 'County', 'PCT_DIABETES_ADULTS13', 'PCT_OBESE_ADULTS13']))
target = 'PCT_OBESE_ADULTS13'




In [24]:
restaurants.columns

Index([u'FIPS', u'State', u'County', u'FFR09', u'FFR14', u'PCH_FFR_09_14',
       u'FFRPTH09', u'FFRPTH14', u'PCH_FFRPTH_09_14', u'FSR09', u'FSR14',
       u'PCH_FSR_09_14', u'FSRPTH09', u'FSRPTH14', u'PCH_FSRPTH_09_14',
       u'PC_FFRSALES07', u'PC_FFRSALES12', u'PC_FSRSALES07', u'PC_FSRSALES12'],
      dtype='object')

In [45]:

dfs = [stores, assistance, insecurity, local, health, restaurants, socioeconomic, prices_taxes]
for df in dfs:
    df.drop(columns=['State', 'County'], axis=1, inplace=True)
    df.set_index('FIPS', inplace=True)



In [46]:
stores.head()

,GROC09,GROC14,PCH_GROC_09_14,GROCPTH09,GROCPTH14,PCH_GROCPTH_09_14,SUPERC09,SUPERC14,PCH_SUPERC_09_14,SUPERCPTH09,...,PCH_SNAPS_12_16,SNAPSPTH12,SNAPSPTH16,PCH_SNAPSPTH_12_16,WICS08,WICS12,PCH_WICS_08_12,WICSPTH08,WICSPTH12,PCH_WICSPTH_08_12
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,6,4,-33.333333,0.110834,0.072209,-34.849716,1,1,0.0,0.018472,...,12.694878,0.674004,0.760911,12.894172,6,5,-16.66667,0.119156,0.090067,-24.412460
1003,24,29,20.833333,0.133775,0.144920,8.331001,6,6,0.0,0.033444,...,43.192771,0.725055,0.949753,30.990390,25,27,8.00000,0.141875,0.141517,-0.252126
1005,5,5,0.000000,0.180786,0.185963,2.863838,0,1,NaN,0.000000,...,0.956938,1.280590,1.354387,5.762745,6,7,16.66667,0.201099,0.257344,27.968330
1007,6,5,-16.666667,0.261540,0.222163,-15.055985,1,1,0.0,0.043590,...,20.512821,0.719122,0.864874,20.267995,6,5,-16.66667,0.277919,0.221268,-20.383970
1009,6,6,0.000000,0.104637,0.103952,-0.654897,1,1,0.0,0.017440,...,23.903509,0.657144,0.815946,24.165470,10,6,-40.00000,0.173028,0.103760,-40.033200


In [47]:
access.set_index('FIPS')

,State,County,LACCESS_POP10,LACCESS_POP15,PCH_LACCESS_POP_10_15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,LACCESS_LOWI10,LACCESS_LOWI15,PCH_LACCESS_LOWI_10_15,...,LACCESS_HISP15,PCT_LACCESS_HISP15,LACCESS_NHASIAN15,PCT_LACCESS_NHASIAN15,LACCESS_NHNA15,PCT_LACCESS_NHNA15,LACCESS_NHPI15,PCT_LACCESS_NHPI15,LACCESS_MULTIR15,PCT_LACCESS_MULTIR15
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,AL,Autauga,18428.439685,17496.693038,-5.056026,33.769657,32.062255,5344.427472,6543.676824,22.439248,...,471.136164,0.863345,86.767975,0.159000,61.169869,0.112092,8.817961,0.016159,482.848633,0.884808
1003,AL,Baldwin,35210.814078,30561.264430,-13.204891,19.318473,16.767489,9952.144027,9886.831137,-0.656270,...,1377.874834,0.755973,212.946378,0.116833,181.649648,0.099662,14.819634,0.008131,1127.696098,0.618712
1005,AL,Barbour,5722.305602,6069.523628,6.067799,20.840972,22.105560,3135.676086,2948.790251,-5.959985,...,509.377525,1.855183,17.096410,0.062266,39.960527,0.145539,8.082376,0.029436,462.382655,1.684025
1007,AL,Bibb,1044.867327,969.378841,-7.224696,4.559753,4.230324,491.449066,596.162829,21.307144,...,8.596762,0.037516,1.994318,0.008703,2.513097,0.010967,0.000000,0.000000,5.259244,0.022951
1009,AL,Blount,1548.175559,3724.428242,140.568857,2.700840,6.497380,609.027708,1650.959482,171.081177,...,497.489891,0.867886,8.428994,0.014705,28.938242,0.050484,1.062851,0.001854,202.914187,0.353990
1011,AL,Bullock,4089.983548,4141.900365,1.269365,37.474652,37.950342,2199.359828,2154.789300,-2.026523,...,24.078889,0.220624,0.968748,0.008876,7.705288,0.070600,0.000000,0.000000,36.523171,0.334645
1013,AL,Butler,1307.819706,1304.328756,-0.266929,6.243470,6.226805,593.225139,612.934761,3.322452,...,11.084902,0.052919,4.955498,0.023657,7.298759,0.034844,0.000000,0.000000,11.440987,0.054619
1015,AL,Calhoun,30901.150585,27785.986765,-10.081061,26.061086,23.433852,11909.162858,10883.209969,-8.614820,...,970.893191,0.818822,235.071203,0.198252,118.296515,0.099768,21.961507,0.018522,886.573464,0.747709
1017,AL,Chambers,6748.213223,7131.505717,5.679911,19.722967,20.843214,3018.284747,3533.916216,17.083593,...,126.786631,0.370559,108.493406,0.317093,20.688146,0.060465,0.061101,0.000179,104.189620,0.304514


In [48]:
access.head()

,FIPS,State,County,LACCESS_POP10,LACCESS_POP15,PCH_LACCESS_POP_10_15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,LACCESS_LOWI10,LACCESS_LOWI15,...,LACCESS_HISP15,PCT_LACCESS_HISP15,LACCESS_NHASIAN15,PCT_LACCESS_NHASIAN15,LACCESS_NHNA15,PCT_LACCESS_NHNA15,LACCESS_NHPI15,PCT_LACCESS_NHPI15,LACCESS_MULTIR15,PCT_LACCESS_MULTIR15
0,1001,AL,Autauga,18428.439685,17496.693038,-5.056026,33.769657,32.062255,5344.427472,6543.676824,...,471.136164,0.863345,86.767975,0.159000,61.169869,0.112092,8.817961,0.016159,482.848633,0.884808
1,1003,AL,Baldwin,35210.814078,30561.264430,-13.204891,19.318473,16.767489,9952.144027,9886.831137,...,1377.874834,0.755973,212.946378,0.116833,181.649648,0.099662,14.819634,0.008131,1127.696098,0.618712
2,1005,AL,Barbour,5722.305602,6069.523628,6.067799,20.840972,22.105560,3135.676086,2948.790251,...,509.377525,1.855183,17.096410,0.062266,39.960527,0.145539,8.082376,0.029436,462.382655,1.684025
3,1007,AL,Bibb,1044.867327,969.378841,-7.224696,4.559753,4.230324,491.449066,596.162829,...,8.596762,0.037516,1.994318,0.008703,2.513097,0.010967,0.000000,0.000000,5.259244,0.022951
4,1009,AL,Blount,1548.175559,3724.428242,140.568857,2.700840,6.497380,609.027708,1650.959482,...,497.489891,0.867886,8.428994,0.014705,28.938242,0.050484,1.062851,0.001854,202.914187,0.353990


In [49]:
stores.head()

,GROC09,GROC14,PCH_GROC_09_14,GROCPTH09,GROCPTH14,PCH_GROCPTH_09_14,SUPERC09,SUPERC14,PCH_SUPERC_09_14,SUPERCPTH09,...,PCH_SNAPS_12_16,SNAPSPTH12,SNAPSPTH16,PCH_SNAPSPTH_12_16,WICS08,WICS12,PCH_WICS_08_12,WICSPTH08,WICSPTH12,PCH_WICSPTH_08_12
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,6,4,-33.333333,0.110834,0.072209,-34.849716,1,1,0.0,0.018472,...,12.694878,0.674004,0.760911,12.894172,6,5,-16.66667,0.119156,0.090067,-24.412460
1003,24,29,20.833333,0.133775,0.144920,8.331001,6,6,0.0,0.033444,...,43.192771,0.725055,0.949753,30.990390,25,27,8.00000,0.141875,0.141517,-0.252126
1005,5,5,0.000000,0.180786,0.185963,2.863838,0,1,NaN,0.000000,...,0.956938,1.280590,1.354387,5.762745,6,7,16.66667,0.201099,0.257344,27.968330
1007,6,5,-16.666667,0.261540,0.222163,-15.055985,1,1,0.0,0.043590,...,20.512821,0.719122,0.864874,20.267995,6,5,-16.66667,0.277919,0.221268,-20.383970
1009,6,6,0.000000,0.104637,0.103952,-0.654897,1,1,0.0,0.017440,...,23.903509,0.657144,0.815946,24.165470,10,6,-40.00000,0.173028,0.103760,-40.033200


In [76]:
# Combine all sheets into one dataframe by joining on FIPS col.

#master_df = []
#for df in dfs:
#    master_df = access.join(df, on="FIPS")
# In [37]: df1.merge(df2, right_index=True, left_on='v2', how='outer')

master_df = access.join(prices_taxes)

In [88]:
access[access['FIPS']==1003].index

Int64Index([1], dtype='int64')

In [89]:
access_test = access
access.set_index('FIPS', inplace=True)

In [90]:
access.head()

,State,County,LACCESS_POP10,LACCESS_POP15,PCH_LACCESS_POP_10_15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,LACCESS_LOWI10,LACCESS_LOWI15,PCH_LACCESS_LOWI_10_15,...,LACCESS_HISP15,PCT_LACCESS_HISP15,LACCESS_NHASIAN15,PCT_LACCESS_NHASIAN15,LACCESS_NHNA15,PCT_LACCESS_NHNA15,LACCESS_NHPI15,PCT_LACCESS_NHPI15,LACCESS_MULTIR15,PCT_LACCESS_MULTIR15
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,AL,Autauga,18428.439685,17496.693038,-5.056026,33.769657,32.062255,5344.427472,6543.676824,22.439248,...,471.136164,0.863345,86.767975,0.159000,61.169869,0.112092,8.817961,0.016159,482.848633,0.884808
1003,AL,Baldwin,35210.814078,30561.264430,-13.204891,19.318473,16.767489,9952.144027,9886.831137,-0.656270,...,1377.874834,0.755973,212.946378,0.116833,181.649648,0.099662,14.819634,0.008131,1127.696098,0.618712
1005,AL,Barbour,5722.305602,6069.523628,6.067799,20.840972,22.105560,3135.676086,2948.790251,-5.959985,...,509.377525,1.855183,17.096410,0.062266,39.960527,0.145539,8.082376,0.029436,462.382655,1.684025
1007,AL,Bibb,1044.867327,969.378841,-7.224696,4.559753,4.230324,491.449066,596.162829,21.307144,...,8.596762,0.037516,1.994318,0.008703,2.513097,0.010967,0.000000,0.000000,5.259244,0.022951
1009,AL,Blount,1548.175559,3724.428242,140.568857,2.700840,6.497380,609.027708,1650.959482,171.081177,...,497.489891,0.867886,8.428994,0.014705,28.938242,0.050484,1.062851,0.001854,202.914187,0.353990


In [91]:
master_df = access.join(dfs)

In [123]:
master_df['County'].dtype

dtype('O')

In [92]:
master_df.head()

,State,County,LACCESS_POP10,LACCESS_POP15,PCH_LACCESS_POP_10_15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,LACCESS_LOWI10,LACCESS_LOWI15,PCH_LACCESS_LOWI_10_15,...,METRO13,POPLOSS10,MILK_PRICE10,SODA_PRICE10,MILK_SODA_PRICE10,SODATAX_STORES14,SODATAX_VENDM14,CHIPSTAX_STORES14,CHIPSTAX_VENDM14,FOOD_TAX14
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,AL,Autauga,18428.439685,17496.693038,-5.056026,33.769657,32.062255,5344.427472,6543.676824,22.439248,...,1,0.0,0.970305,0.972216,0.923289,4.0,4.0,4.0,4.0,4.0
1003,AL,Baldwin,35210.814078,30561.264430,-13.204891,19.318473,16.767489,9952.144027,9886.831137,-0.656270,...,1,0.0,1.017690,1.001391,0.940165,4.0,4.0,4.0,4.0,4.0
1005,AL,Barbour,5722.305602,6069.523628,6.067799,20.840972,22.105560,3135.676086,2948.790251,-5.959985,...,0,0.0,1.136671,0.992584,1.059400,4.0,4.0,4.0,4.0,4.0
1007,AL,Bibb,1044.867327,969.378841,-7.224696,4.559753,4.230324,491.449066,596.162829,21.307144,...,1,0.0,0.970305,0.972216,0.923289,4.0,4.0,4.0,4.0,4.0
1009,AL,Blount,1548.175559,3724.428242,140.568857,2.700840,6.497380,609.027708,1650.959482,171.081177,...,1,0.0,0.970305,0.972216,0.923289,4.0,4.0,4.0,4.0,4.0


In [94]:
df_cols = list(set(features_new) - set(['FIPS']))

# Create a dataframe with just the cols of interest.
df = master_df[df_cols]
df.shape

(3143, 58)

In [102]:
features_new

['FIPS',
 'State',
 'County',
 'GROCPTH14',
 'SUPERCPTH14',
 'CONVSPTH14',
 'SPECSPTH14',
 'SNAPSPTH16',
 'FIPS',
 'State',
 'County',
 'PCT_SNAP16',
 'PCT_NSLP15',
 'PCT_SBP15',
 'PCT_SFSP15',
 'PCT_WIC15',
 'FIPS',
 'State',
 'County',
 'FMRKTPTH16',
 'PCT_FMRKT_SNAP16',
 'PCT_FMRKT_WIC16',
 'PCT_FMRKT_WICCASH16',
 'PCT_FMRKT_SFMNP16',
 'PCT_FMRKT_CREDIT16',
 'PCT_FMRKT_FRVEG16',
 'PCT_FMRKT_ANMLPROD16',
 'PCT_FMRKT_BAKED16',
 'PCT_FMRKT_OTHERFOOD16',
 'FOODHUB16',
 'FIPS',
 'State',
 'County',
 'PCT_DIABETES_ADULTS13',
 'PCT_OBESE_ADULTS13',
 'PCT_HSPA15',
 'RECFACPTH14',
 'FIPS',
 'State',
 'County',
 'FFRPTH14',
 'FSRPTH14',
 'FIPS',
 'State',
 'County',
 'PCT_NHWHITE10',
 'PCT_NHBLACK10',
 'PCT_HISP10',
 'PCT_NHASIAN10',
 'PCT_NHNA10',
 'PCT_NHPI10',
 'PCT_65OLDER10',
 'PCT_18YOUNGER10',
 'MEDHHINC15',
 'POVRATE15',
 'METRO13',
 'FIPS',
 'State',
 'County',
 'SODATAX_STORES14',
 'SODATAX_VENDM14',
 'CHIPSTAX_STORES14',
 'CHIPSTAX_VENDM14',
 'FOOD_TAX14',
 'FIPS',
 'State',
 'Coun

In [101]:
df.head()

,PCT_OBESE_ADULTS13,PCT_SNAP16,PCT_FMRKT_OTHERFOOD16,PCT_DIABETES_ADULTS13,PCT_SFSP15,PCT_NHNA10,PCT_SBP15,SODATAX_STORES14,CHIPSTAX_VENDM14,State,...,FOOD_TAX14,PCT_NHPI10,MEDHHINC15,County,SPECSPTH14,PCT_NHASIAN10,PCT_FMRKT_SFMNP16,PCT_LACCESS_HISP15,PCT_LACCESS_POP15,PCT_LACCESS_BLACK15
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,34.1,16.976436,100.0,13.0,1.027109,0.397647,5.232998,4.0,4.0,AL,...,4.0,0.040314,56580.0,Autauga,0.036104,0.855766,0.0,0.863345,32.062255,7.726582
1003,27.4,16.976436,75.0,10.4,1.027109,0.628755,5.232998,4.0,4.0,AL,...,4.0,0.043343,52387.0,Baldwin,0.129928,0.735193,100.0,0.755973,16.767489,1.942757
1005,44.4,16.976436,50.0,18.4,1.027109,0.218524,5.232998,4.0,4.0,AL,...,4.0,0.087409,31433.0,Barbour,0.074385,0.389700,50.0,1.855183,22.105560,9.881360
1007,40.3,16.976436,0.0,14.8,1.027109,0.279293,5.232998,4.0,4.0,AL,...,4.0,0.030548,40767.0,Bibb,0.044433,0.096007,0.0,0.037516,4.230324,1.828933
1009,34.6,16.976436,100.0,14.1,1.027109,0.497191,5.232998,4.0,4.0,AL,...,4.0,0.031402,50487.0,Blount,0.000000,0.200621,100.0,0.867886,6.497380,0.167201


In [95]:
# Check for null values...
df.isnull().sum()

PCT_OBESE_ADULTS13          1
PCT_SNAP16                  0
PCT_FMRKT_OTHERFOOD16     895
PCT_DIABETES_ADULTS13       1
PCT_SFSP15                  0
PCT_NHNA10                  0
PCT_SBP15                   0
SODATAX_STORES14            0
CHIPSTAX_VENDM14            0
State                       0
PCT_NHWHITE10               0
FFRPTH14                    0
FSRPTH14                    0
PCT_FMRKT_WIC16           895
METRO13                     0
GROCPTH14                   0
PCT_LACCESS_NHPI15         19
RECFACPTH14                 0
PCT_HISP10                  0
PCT_FMRKT_BAKED16         895
PCT_LACCESS_SNAP15         20
PCT_FMRKT_WICCASH16       895
SODATAX_VENDM14             0
PCT_LACCESS_NHASIAN15      19
SNAPSPTH16                 29
PCT_WIC15                   0
PCT_LACCESS_SENIORS15      19
FMRKTPTH16                  2
POVRATE15                   4
SUPERCPTH14                 0
PCT_FMRKT_FRVEG16         895
PCT_HSPA15               1118
PCT_FMRKT_SNAP16          895
PCT_65OLDE

In [96]:
# ...and drop them
df = df.dropna()
df.isnull().sum()

PCT_OBESE_ADULTS13       0
PCT_SNAP16               0
PCT_FMRKT_OTHERFOOD16    0
PCT_DIABETES_ADULTS13    0
PCT_SFSP15               0
PCT_NHNA10               0
PCT_SBP15                0
SODATAX_STORES14         0
CHIPSTAX_VENDM14         0
State                    0
PCT_NHWHITE10            0
FFRPTH14                 0
FSRPTH14                 0
PCT_FMRKT_WIC16          0
METRO13                  0
GROCPTH14                0
PCT_LACCESS_NHPI15       0
RECFACPTH14              0
PCT_HISP10               0
PCT_FMRKT_BAKED16        0
PCT_LACCESS_SNAP15       0
PCT_FMRKT_WICCASH16      0
SODATAX_VENDM14          0
PCT_LACCESS_NHASIAN15    0
SNAPSPTH16               0
PCT_WIC15                0
PCT_LACCESS_SENIORS15    0
FMRKTPTH16               0
POVRATE15                0
SUPERCPTH14              0
PCT_FMRKT_FRVEG16        0
PCT_HSPA15               0
PCT_FMRKT_SNAP16         0
PCT_65OLDER10            0
CHIPSTAX_STORES14        0
PCT_NSLP15               0
PCT_18YOUNGER10          0
P

In [97]:
# make sure metro is boolean
df['METRO13'] = df['METRO13'].astype('bool')

u"FIPS\n1001                  Autauga\n1003                  Baldwin\n1005                  Barbour\n1007                     Bibb\n1009                   Blount\n1011                  Bullock\n1013                   Butler\n1015                  Calhoun\n1017                 Chambers\n1019                 Cherokee\n1023                  Choctaw\n1025                   Clarke\n1027                     Clay\n1029                 Cleburne\n1031                   Coffee\n1033                  Colbert\n1035                  Conecuh\n1039                Covington\n1041                 Crenshaw\n1043                  Cullman\n1045                     Dale\n1047                   Dallas\n1049                   DeKalb\n1051                   Elmore\n1053                 Escambia\n1055                   Etowah\n1057                  Fayette\n1059                 Franklin\n1063                   Greene\n1065                     Hale\n1067                    Henry\n1069                  Houston\n

In [134]:
df['County'] = df['County'].astype('category').encode('utf-8')

AttributeError: 'Series' object has no attribute 'encode'

In [142]:
print df[df['County'] == 'Orange'].County

Series([], Name: County, dtype: category
Categories (1, object): [FIPS\n1001                  Autauga\n1003     ...])


In [126]:
df['State'] = df['State'].astype('category')

In [127]:
# dtypes look good
print(df.dtypes)

PCT_OBESE_ADULTS13        float64
PCT_SNAP16                float64
PCT_FMRKT_OTHERFOOD16     float64
PCT_DIABETES_ADULTS13     float64
PCT_SFSP15                float64
PCT_NHNA10                float64
PCT_SBP15                 float64
SODATAX_STORES14          float64
CHIPSTAX_VENDM14          float64
State                    category
PCT_NHWHITE10             float64
FFRPTH14                  float64
FSRPTH14                  float64
PCT_FMRKT_WIC16           float64
METRO13                      bool
GROCPTH14                 float64
PCT_LACCESS_NHPI15        float64
RECFACPTH14               float64
PCT_HISP10                float64
PCT_FMRKT_BAKED16         float64
PCT_LACCESS_SNAP15        float64
PCT_FMRKT_WICCASH16       float64
SODATAX_VENDM14           float64
PCT_LACCESS_NHASIAN15     float64
SNAPSPTH16                float64
PCT_WIC15                 float64
PCT_LACCESS_SENIORS15     float64
FMRKTPTH16                float64
POVRATE15                 float64
SUPERCPTH14   

In [128]:
df.shape

(1540, 58)

In [129]:
df.head()

,PCT_OBESE_ADULTS13,PCT_SNAP16,PCT_FMRKT_OTHERFOOD16,PCT_DIABETES_ADULTS13,PCT_SFSP15,PCT_NHNA10,PCT_SBP15,SODATAX_STORES14,CHIPSTAX_VENDM14,State,...,FOOD_TAX14,PCT_NHPI10,MEDHHINC15,County,SPECSPTH14,PCT_NHASIAN10,PCT_FMRKT_SFMNP16,PCT_LACCESS_HISP15,PCT_LACCESS_POP15,PCT_LACCESS_BLACK15
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,34.1,16.976436,100.0,13.0,1.027109,0.397647,5.232998,4.0,4.0,AL,...,4.0,0.040314,56580.0,FIPS\n1001 Autauga\n1003 ...,0.036104,0.855766,0.0,0.863345,32.062255,7.726582
1003,27.4,16.976436,75.0,10.4,1.027109,0.628755,5.232998,4.0,4.0,AL,...,4.0,0.043343,52387.0,FIPS\n1001 Autauga\n1003 ...,0.129928,0.735193,100.0,0.755973,16.767489,1.942757
1005,44.4,16.976436,50.0,18.4,1.027109,0.218524,5.232998,4.0,4.0,AL,...,4.0,0.087409,31433.0,FIPS\n1001 Autauga\n1003 ...,0.074385,0.389700,50.0,1.855183,22.105560,9.881360
1007,40.3,16.976436,0.0,14.8,1.027109,0.279293,5.232998,4.0,4.0,AL,...,4.0,0.030548,40767.0,FIPS\n1001 Autauga\n1003 ...,0.044433,0.096007,0.0,0.037516,4.230324,1.828933
1009,34.6,16.976436,100.0,14.1,1.027109,0.497191,5.232998,4.0,4.0,AL,...,4.0,0.031402,50487.0,FIPS\n1001 Autauga\n1003 ...,0.000000,0.200621,100.0,0.867886,6.497380,0.167201


In [133]:
df.to_csv('data/df_for_webapp.csv', index=False)

UnicodeEncodeError: 'ascii' codec can't encode character u'\xf1' in position 27508: ordinal not in range(128)